# Homework 01

Please fill in your name: Morgan Schalizki

## Question 0

Fill in your name at the top and complete the instructions for the questions below. You need to fill in blanks ________ and write code (sections often contain a comment that begins with "TODO", which you should remove). When you are done, download the file as a .ipynb and submit on Canvas.

Before starting with this assignment, open and work through "python-intro.ipynb" and read up on things you don't understand. See the Python Tutorial PDF on Canvas for more information.

## Question 1
I demonstrated in class that the integer data type in Python is not a typical 32 or 64 bit integer with a largest value of $2^{32}$ or $2^{64}$ but basically unlimited in size.

Find out the format Python is using to store floating point numbers like 3.4 (unlimited precision? What IEEE floating point standard is used, 32 or 64 or something else? Write your answer below) and do a computation that demonstrates that this is true.    

In [80]:
import sys
x=3.4


# the number format "float" is known as: double with 64 bits in total


# 52 bits for the number
# 1 for the sign
# and 11 bits for the exponent
# https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter09.02-Floating-Point-Numbers.html
# multiple sources confer with above

# a python object of type float consumes how many bytes? 24 bytes (where 8 are the decimals and 16 for python object)(?)

# Show that your statement above is correct using some expression or
# computation that shows the precision of the underlying format:


# We get 53 bits of precision or 16/17 digits stored 

print(sys.float_info)

print(f"1/10 with 55 digits: {1/10:.55f}")
print(f"1/10 with 60 digits: {1/10:.60f}")
s=3602879701896397 / 2 ** 55
# Note 3602879701896397 is 16 digits
print(f"3602879701896397 / 2 ** 55 = {'{0:.60f}'.format(s)}")

float_size = 1/10 + 1/3
print(f"1/10 + 1/3 = {float_size}")
print(f"A float takes up {sys.getsizeof(float_size)} bytes")
print(f"there are {len(str(float_size))-2} digits in 1/10 + 1/3 in python") # removing first 0 and decimal point
print(f"1/3 is stored as: {1/3} in python")
print(f"there are {len(str(1/3))-2} digits in 1/3 in python")


# While 1/10, 1/3 can yield up to 55/6 digits, we still only have 53 bits
# https://docs.python.org/3/tutorial/floatingpoint.html#floating-point-arithmetic-issues-and-limitations
# Explained in python docs, but my short understanding is that
# due to IEEE754 standards we use 53 bits and then when calculating N for J ~= 2 ** N / 10
# we find an upper bound roughly where N <= 56


# Bonus: why do you think Python uses a fixed size floating point
# format, but has its own unlimited int format?
#
# Suggestion 1: (Large Data)
# There is a tendency to have large data sets and such to loop over and those are indexed by ints
# Thus ints have more functionality for large data along with being computational more often than floats
# (+) anything we need a float for we could do with ints and then convert back to a float if needed
# So we save floats for special cases of small data
#   
# Suggestion 2: (Hardware limitations)
# Probably due to computers having an easier time computing ints
# When doing floating point math we'd have to convert to int or do other math (thus making floats specially stored values)
# which yields a loss since computer compute in binary and ints can be directly computed in binary
# Thus a fixed size is to ease computational burden and memory storage (thus simply a hardware constraint)
# Irrational numbers are such numbers that would overgo our maximal memory allowance if allowed.
# Ints (unless forced x += 1 in an infinite loop) don't require infinite memory, but floats for 1/3 would
# require infinite memory as we'd have an infinite number of 3's in 0.3333... but computers are finite in storage so we cap
# the number of digits
#
# Suggestion 3: IEEE754 + python:
# All decimals/floats are stored in binary/base 2, which makes it extremely difficult to store decimals.

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)
1/10 with 55 digits: 0.1000000000000000055511151231257827021181583404541015625
1/10 with 60 digits: 0.100000000000000005551115123125782702118158340454101562500000
3602879701896397 / 2 ** 55 = 0.100000000000000005551115123125782702118158340454101562500000
1/10 + 1/3 = 0.43333333333333335
A float takes up 24 bytes
there are 17 digits in 1/10 + 1/3 in python
1/3 is stored as: 0.3333333333333333 in python
there are 16 digits in 1/3 in python


## Question 2
Write a function hey_ho() with arguments start and end that prints the numbers starting from start to end (inclusive) in a single line each. If the number is divisible by 3, print "hey!" instead of the number, if the number is divisible by 5 print "ho!". Finally, if it is divisible by both, print "GO Tigers!" instead.
Example for hey_ho(7,16):

In [81]:
def hey_ho(start, end):
    for i in range(start,end+1):
        if (i) % 3==0 and (i) % 5 == 0:
            print("GO Tigers!")
        elif (i) % 3 == 0:
            print("hey!")
        elif (i) % 5 == 0:
            print("ho!")
        else:
            print(i)
        
    
hey_ho(7,15)

7
8
hey!
ho!
11
hey!
13
14
GO Tigers!


# Question 3

We discussed that the size of the number of allocated entries inside a Python "list" often is larger than absolutely necessary. Consider the case of adding one element at a time to an empty list x until it contains 100 elements. Based on the code in "python-list-layout.ipynb", print the final allocated size and count how many allocations happen during the computation by comparing the allocated size to the previous allocated size after each element is added.

In [82]:
import ctypes

class ListStruct(ctypes.Structure): # struct PyListObject
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong),
                ("ob_item", ctypes.c_long),  # PyObject** pointer cast to long
                ("allocated", ctypes.c_ulong)]

x = []

info = ListStruct.from_address(id(x))
reallocations = 0
old_allocated = info.allocated
for i in range(100):
    x.append(1)
    info = ListStruct.from_address(id(x))
    new_allocated = info.allocated
    if(new_allocated != old_allocated):
        old_allocated = new_allocated
        reallocations +=1
info = ListStruct.from_address(id(x))
new_allocated = info.allocated
if(new_allocated != old_allocated):
    reallocations += 1

print(f"There are {info.ob_size} 1's in our array with an allocated size of {info.allocated}")
print(f"Reallocated {reallocations} times once one object (1's) was appended, otherwise 12 times once we added the first object.")
print("i.e. we have 11 allocations starting from [1] and 12 starting from []")


There are 100 1's in our array with an allocated size of 2932775568
Reallocated 11 times once one object (1's) was appended, otherwise 12 times once we added the first object.
i.e. we have 11 allocations starting from [1] and 12 starting from []
